In [1]:
import os
import sys
import json
import argparse

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from glob import glob
from nilearn import plotting

## nilearn modeling: first level


In [2]:
# based on: https://nilearn.github.io/auto_examples/04_glm_first_level/plot_bids_features.html#sphx-glr-auto-examples-04-glm-first-level-plot-bids-features-py

def prep_models_and_args(subject_id=None, 
                         task_label=None, 
                         fwhm=None, 
                         bidsroot=None, 
                         deriv_dir=None, 
                         event_type=None, 
                         t_r=None, t_acq=None, 
                         space_label='MNI152NLin2009cAsym'):
    from nilearn.glm.first_level import first_level_from_bids
    from nilearn.interfaces.fmriprep import load_confounds

    # correct the fmriprep-given slice reference (middle slice, or 0.5)
    slice_time_ref = 0.5 * t_acq / t_r

    print(bidsroot, task_label, space_label)

    models, models_run_imgs, \
            models_events, \
            models_confounds = first_level_from_bids(bidsroot, 
                                                     task_label, 
                                                     space_label,
                                                     [subject_id],
                                                     smoothing_fwhm=fwhm,
                                                     derivatives_folder=deriv_dir,
                                                     slice_time_ref=slice_time_ref,
                                                     minimize_memory=False)


    ''' create events '''
    # stimulus events
    if event_type == 'stimulus':
        for sx, sub_events in enumerate(models_events):
            for mx, run_events in enumerate(sub_events):

                name_groups = run_events.groupby('trial_type')['trial_type']
                suffix = name_groups.cumcount() + 1
                repeats = name_groups.transform('size')

                run_events['trial_type'] = run_events['trial_type']
                run_events['trial_type'] = run_events['trial_type'].str.replace('-','_')

        # create stimulus list from updated events.tsv file
        stim_list = sorted([s for s in run_events['trial_type'].unique() if str(s) != 'nan'])
    
    # trial-specific events
    if event_type == 'trial':
        for sx, sub_events in enumerate(models_events):
            for mx, run_events in enumerate(sub_events):

                name_groups = run_events.groupby('trial_type')['trial_type']
                suffix = name_groups.cumcount() + 1
                repeats = name_groups.transform('size')

                run_events['trial_type'] = run_events['trial_type'] + \
                                                    '_trial' + suffix.map(str)
                run_events['trial_type'] = run_events['trial_type'].str.replace('-','_')

        # create stimulus list from updated events.tsv file
        stim_list = sorted([s for s in run_events['trial_type'].unique() if str(s) != 'nan'])

    # all sound events
    elif event_type == 'sound':
        for sx, sub_events in enumerate(models_events):
            for mx, run_events in enumerate(sub_events):
                orig_stim_list = sorted([str(s) for s in run_events['trial_type'].unique() if str(s) not in ['nan', 'None']])
                #print('original stim list: ', orig_stim_list)

                run_events['trial_type'] = run_events.trial_type.str.split('_', expand=True)[0]

        # create stimulus list from updated events.tsv file
        stim_list = sorted([str(s) for s in run_events['trial_type'].unique() if str(s) not in ['nan', 'None']])
        #print('stim list: ', stim_list)

    #model_and_args = zip(models, models_run_imgs, models_events, models_confounds)
    return stim_list, models, models_run_imgs, models_events, models_confounds



In [3]:
# ### Across-runs GLM
def nilearn_glm_across_runs(stim_list, task_label, 
                            models, models_run_imgs, 
                            models_events, models_confounds, 
                            space_label):
    from nilearn.reporting import make_glm_report
    from nilearn.interfaces.bids import save_glm_to_bids
    from nilearn.interfaces.fmriprep import load_confounds_strategy
    
    for midx in range(len(models)):
        #for sx, stim in enumerate(stim_list):
        
        for contrast_label in ['sound', 'response']:
            #contrast_desc  = stim
            print('Running for contrast', contrast_label)
            
            midx = 0 # only 1 subject per analysis
            model = models[midx]
            imgs = models_run_imgs[midx]
            events = models_events[midx]
            #confounds = models_confounds[midx]

            # set limited confounds
            print('selecting confounds')
            confounds_ltd, sample_mask = load_confounds_strategy(img_files=imgs, 
                                                                 denoise_strategy='compcor')
            
            try:
                # fit the GLM
                print('fitting GLM')
                model.fit(imgs, events, confounds_ltd); 
                print(model)

                # compute the contrast of interest
                print('computing contrast of interest')
                summary_statistics = model.compute_contrast(contrast_label, output_type='all')
                zmap = summary_statistics['z_score']
                tmap = summary_statistics['stat']
                statmap = summary_statistics['effect_size']

                ''' ADD BIDS OUTPUTS '''
                bidsderiv_dir = os.path.join(bidsroot, 'derivatives', 
                                             'nilearn', 'run-all')
                if not os.path.exists(bidsderiv_dir):
                    os.makedirs(bidsderiv_dir)

                out_prefix = f"sub-{model.subject_label}_task-{task_label}_fwhm-{model.smoothing_fwhm}"
                save_glm_to_bids(model, 
                                 contrast_label,
                                 out_dir=bidsderiv_dir,
                                 prefix=out_prefix,
                                )
                print(f'Saved model outputs to {bidsderiv_dir}')

            except:
                print('could not run for ', contrast_label)
        return confounds_ltd#summary_statistics


## Run pipelines

In [4]:
task_label = 'badaga'
t_acq = 2
t_r = t_acq # no silent gap in this acquisition

project_dir = os.path.join('/bgfs/bchandrasekaran/krs228/data/', 
                           'SSP/')
bidsroot = os.path.join(project_dir, 
                        'data_bids')
fmriprep_dir = os.path.join(bidsroot, 
                            'derivatives', 
                            'fmriprep-23.2.1',
                            )
print('bidsroot: ', bidsroot)
print('fmriprep dir:', fmriprep_dir)

nilearn_dir = os.path.join(bidsroot, 'derivatives', 'nilearn')
if not os.path.exists(nilearn_dir):
        os.makedirs(nilearn_dir)

bidsroot:  /bgfs/bchandrasekaran/krs228/data/SSP/data_bids
fmriprep dir: /bgfs/bchandrasekaran/krs228/data/SSP/data_bids/derivatives/fmriprep-23.2.1


### Univariate analysis

Takes approximately 30 minutes per subject

**from Ashley Feb 3, 2025:**

So, please IGNORE for now:
- 001
- 002
- 005
- 014
- 072
- (and maybe 069?)

In [11]:
fwhm = 6
space_label = 'MNI152NLin2009cAsym'
subject_list = [#'SSP009', 'SSP011', 
                #'SSP012', 'SSP013', # only 2 beh events files ?
                #'SSP017', 'SSP018',
                #'SSP020', 'SSP028', 'SSP032', 'SSP033',
                #'SSP034', 'SSP036', 'SSP038', 'SSP039',
                #'SSP040', 
                #'SSP041', 'SSP038', 'SSP045',
                #'SSP046', 'SSP048', 'SSP051', 'SSP054',
                #'SSP058', 'SSP059', 'SSP069',
                ] # 


In [12]:
for sx, subject_id in enumerate(subject_list):
    print('Running subject ', subject_id)
    stim_list, models, \
               models_run_imgs, \
               models_events, \
               models_confounds = prep_models_and_args(subject_id, task_label, 
                                                     fwhm, bidsroot, 
                                                     fmriprep_dir, 'sound',
                                                     t_r, t_acq, 
                                                     space_label)
   
    # Across-run GLM
    confounds_ltd = nilearn_glm_across_runs(stim_list, task_label, 
                                             models, models_run_imgs, 
                                             models_events, models_confounds, 
                                             space_label)


Running subject  SSP069
/bgfs/bchandrasekaran/krs228/data/SSP/data_bids badaga MNI152NLin2009cAsym


ValueError: No events.tsv files found for subject SSP069 for filter: [('task', 'badaga')].

## tSNR estimates

In [4]:
# ## plot tsnr
def plot_tsnr(bold_files):
    from nilearn import image

    thresh = 0
    fwhm = 5

    for fx,filepath in enumerate(bold_files):
        tsnr_func = image.math_img('img.mean(axis=3) / img.std(axis=3)', img=filepath)
        tsnr_func_smooth = image.smooth_img(tsnr_func, fwhm=5)

        display = plotting.plot_stat_map(tsnr_func_smooth, 
                                        bg_img=t1w_fpath, 
                                        #title='fMRI single run tSNR map',
                                        #cut_coords=[8,50,-20],
                                        #threshold=thresh, 
                                        #cmap='jet'
                                        );
        display.show()
        

In [ ]:
from nilearn import image
space_label = 'MNI152NLin2009cAsym'
fwhm = 1.5
tsnr_smooth = 3


In [ ]:
sub_list_mand = ['FLT01', 'FLT03', 'FLT05', 'FLT07', 'FLT08', 'FLT10', ] # 

sub_list_nman = ['FLT02', 'FLT04', 'FLT06', 'FLT09', 'FLT11', 'FLT12', 'FLT13', ]

sub_list = sub_list_mand + sub_list_nman

In [ ]:
roi_list = ['L-CN', 'L-SOC', 'L-IC', 'L-MGN', 'L-HG', 'L-PP', 'L-PT', 'L-STGp', 'L-STGa', 
            'R-CN', 'R-SOC', 'R-IC', 'R-MGN', 'R-HG', 'R-PP', 'R-PT', 'R-STGp', 'R-STGa', ]

In [ ]:
data_list_sub = []
data_list_roi = []
data_list_run = []
data_list_tsnr = []

for sx, subject_id in enumerate(sub_list):
    print('Running subject ', subject_id)
    # Univariate analysis: MNI space, 3 mm, across-run GLM
    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, 
                                                                                 fwhm, bidsroot, 
                                                                                 deriv_dir, 'sound',
                                                                                 t_r, t_acq, space_label)
    for mx, mask_descrip in enumerate(roi_list):
        #mask_descrip = 'L-STGp'
        print(mask_descrip)
        masks_dir = os.path.join(nilearn_dir, 'masks', 
                                 'sub-%s'%subject_id, 
                                 'space-%s'%space_label) #'masks-aparc' # 'masks-dseg' , 'masks-subcort-aud'
        mask_fpath = glob(masks_dir + '/*/' + 'sub-{}_space-{}_mask-{}.nii.gz'.format(subject_id, space_label, mask_descrip))[0]

        for px, preproc_fpath in enumerate(models_run_imgs[0]):
            tsnr_func = image.math_img('img.mean(axis=3) / img.std(axis=3)', img=preproc_fpath);
            tsnr_func_smooth = image.smooth_img(tsnr_func, fwhm=tsnr_smooth)

            #display = plotting.plot_epi(tsnr_func_smooth, colorbar=True, vmin=10, vmax=100,
            #                            title=os.path.basename(preproc_fpath));

            fmri_masked, masker = mask_fmri(tsnr_func_smooth, mask_fpath, fwhm)
            print(np.mean(fmri_masked))

            data_list_sub.append(subject_id)
            data_list_roi.append(mask_descrip)
            data_list_run.append(px+1)
            data_list_tsnr.append(np.mean(fmri_masked))

In [ ]:
print(len(data_list_sub))
print(len(data_list_roi))
print(len(data_list_run))
print(len(data_list_tsnr))


In [ ]:
np.unique(data_list_roi)

In [ ]:
np.unique(data_list_sub)

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
tsnr_df = pd.DataFrame({'sub_id': data_list_sub, 'Region': data_list_roi, 'run': data_list_run, 'tSNR': data_list_tsnr})
tsnr_df['hemi'] = tsnr_df.Region.str[0]
tsnr_df.Region = tsnr_df.Region.str[2:]

In [ ]:
tsnr_df

In [ ]:
mean_tsnr_df = tsnr_df.groupby(['sub_id', 'Region', 'hemi'], as_index=False, sort=False)['tSNR'].mean()

In [ ]:
mean_tsnr_df.columns

In [ ]:
mean_tsnr_df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4), dpi=300)
sns.swarmplot(data=tsnr_df, x='Region', y='tSNR', hue='hemi')
fig.suptitle('temporal SNR by region of interest')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4), dpi=300)
sns.violinplot(data=mean_tsnr_df, x='Region', y='tSNR', hue='hemi')
fig.suptitle('temporal SNR by region of interest')